<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''
select 
count(v.id)
from public.VACANCIES v
'''

In [5]:
# результат запроса
result = pd.read_sql_query(query_3_1, connection)
result.iloc[0][0]

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
select 
count(v.id)
from public.EMPLOYERS v
'''

In [7]:
# результат запроса
result = pd.read_sql_query(query_3_2, connection)
result.iloc[0][0]

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
select 
count(a.id)
from public.areas a
'''

In [9]:
# результат запроса
result = pd.read_sql_query(query_3_3, connection)
result.iloc[0][0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
select 
count(i.id)
from public.INDUSTRIES i
'''

In [11]:
# результат запроса
result = pd.read_sql_query(query_3_4, connection)
result.iloc[0][0]

294

***
### Выводы по предварительному анализу вакансий:

*По результатам предварительного анализа даных можно сделать вывод о обширном, но не всеобемлющем охвате как регионов
так и сфер деятельности. Обращает на себя внимание тот факт, что количество работодателей лишь в 2 раза меньше чем вакансий. Что может говорить о не слишком большом спектре вакансий у каждого из работодателей.*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
select 
count(v.id),  -- считаем количество вакансий
a.name
from public.VACANCIES v
--объединяем таблицы вакансий и регионов по идентификаторам
join public.areas a on a.id=v.area_id                   
-- группируем по наименованию региона
group by a.name
-- отсортируем по количеству вакансий в порядке убывания
order by 1 desc  
-- выведем 5 первых значений                        
limit 5         
'''

In [13]:
# результат запроса
result = pd.read_sql_query(query_4_1, connection)
list(result['name'])

['Москва', 'Санкт-Петербург', 'Минск', 'Новосибирск', 'Алматы']

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
select 
-- считаем количество вакансий
count(v.id)   
from public.VACANCIES v
-- условием отбора делаем наличие данных в одном из
-- двух полей с зарплатой минимальной и максимальной
where v.salary_from is not null or v.salary_to is not null  
                                                            
'''

In [15]:
# результат запроса
result = pd.read_sql_query(query_4_2, connection)
result.iloc[0][0]    

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
select 
--считаем среднее значение для минимальной зарплаты
round(avg(v.salary_from),0) min,   
--считаем среднее значение для максимальной зарплаты
round(avg(v.salary_to),0) max                          
from public.VACANCIES v
'''

In [17]:
# результат запроса
result = pd.read_sql_query(query_4_3, connection)
result.iloc[0]

min     71065.0
max    110537.0
Name: 0, dtype: float64

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
select 
-- считаем количество вакансий
count(v.id),   
-- находим комбинацию типа рабочего графика и 
-- типа трудоустройства
v.schedule||'-'||v.employment combination  
from public.VACANCIES v
-- группируем по сочетанию типа рабочего графика 
-- и типа трудоустройства
group by 2           
-- сотрируем по количеству найденных вакансий для 
-- получившихся сочетаний по убыванию                   
order by 1 desc      
-- берем две первые записи                    
limit 2             
'''

In [19]:
# результат запроса
result = pd.read_sql_query(query_4_4, connection)
result

,count,combination
0,35367,Полный день-Полная занятость
1,7802,Удаленная работа-Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
select 
-- считаем количество вакансий
count(v.id),  
v.experience
from public.VACANCIES v
-- группируем по значению поля Требуемый опыт работы
group by 2    
-- сотритуем по количеству найденных вакансий для каждого варинта
order by 1       
'''

In [21]:
# результат запроса
result = pd.read_sql_query(query_4_5, connection)
list(result['experience'])

['Более 6 лет', 'Нет опыта', 'От 3 до 6 лет', 'От 1 года до 3 лет']

***
### Выводы по детальному анализу вакансий


- *Количество вакансий коррелирует с населением и промышленно-деловым потенциалом городов.*
- *Не многим более чем у половины вакансий заполнены поля с ожидаемым уровнем зарплаты, что может говорить, для части работодателей, как об ориентировании этой группы на средние значения по рынку так и не желании сообщать эту информацию до проведения собеседования.*
- *Диапазон колебаний средней зарплаты в районе 25% говорит о достаочно стабильном её уровне, при условии различного уровня зарплат в различных регионах.*
- *Соотношение типов рабочего графика говорит о преобладающих тенденциях поиска работодателями, опубликовавшими вакансии, работников для работы в офисе.*
- *Динамика количества вакансий в зависимости от опыта может говорить о том, что площадка НН не слишком активно используется для поиска сотрудников с большим опытом, которые вероятно имеют возможность искать работу с использованием прямых контактов. А используется преимуществено для поиска выпускников учебных заведений с не большим стажем работы.*

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''
-- находим работодателя на первом месте по количеству вакансий
(select  
-- считаем количество вакансий
count(v.id), 
e.name
from public.VACANCIES v
--объединяем таблицы вакансий и работодателей по ключевым полям
join public.EMPLOYERS e on e.id = v.employer_id  
-- группируем по наименованиям работодателей            
group by 2 
-- сортируем по возрастанию по количеству вакансий
order by 1 desc 
-- выводим первую запись
limit 1) 
-- соединяем таблицы по вертикали
union 
-- находим работодателя на 5-м месте по количеству вакансий
(select 
-- считаем количество вакансий
count(v.id),  
e.name
from public.VACANCIES v
-- объединяем таблицы вакансий и работодателей по ключевым полям
join public.EMPLOYERS e on e.id = v.employer_id   
-- группируем по наименованиям работодателей                
group by 2        
-- сортируем по возрастанию по количеству вакансий 
order by 1 desc   
-- пропускаем 4 первых значения
offset 4     
-- берем 5-е значение
limit 1)     
-- сортируем полученный результат по убыванию
order by 1 desc  
'''

In [23]:
# результат запроса
result = pd.read_sql_query(query_5_1, connection)
print('-'.join(list(result['name'])))

Яндекс-Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''
select 
a.name,
-- считаем количество работоателей
count(e.id) работодатели, 
-- считаем количесво вакансий
count(v.id) вакансии      
from public.AREAS a
-- объединяем таблицы работодателей и регионов
join public.EMPLOYERS e on a.id = e.area       
-- объединяем таблицы вакансий и регионов
left join public.VACANCIES v on a.id = v.area_id  
-- группироуем по наименованию региона
group by 1             
-- ограничиваем отбор регионами в которых нет 
-- вакансий (их количество равно нулю)
having count(v.id) = 0 
-- упрядочим по количесву работодателей по убыванию
order by 2 desc           
-- берем первую запись
limit 1                   
''' 

In [25]:
# результат запроса
result = pd.read_sql_query(query_5_2, connection)
print(result['name'][0])

Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''
select 
e.name,
-- считаем количесво уникальных регионов в которых 
-- работодатели публикуют вакансии
count(distinct(v.area_id)) 
from public.EMPLOYERS e
-- объединем таблицы вакансий и работодателей
left join public.VACANCIES v on e.id = v.employer_id 
-- группируем по наименованию работодателя
group by 1               
-- сортируем по количеству регионов по убыванию
order by 2 desc            
'''

In [27]:
# результат запроса
result = pd.read_sql_query(query_5_3, connection)
result

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23170,СДЕЛКА,0
23171,Alandr Group,0
23172,СДК,0
23173,Lemon Land Lombard,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''
select 
-- считаем количество работодателей
count(e.name)         
from public.EMPLOYERS e 
-- объединяем промежуточную таблицу и таблицу рабтодателей
left join public.EMPLOYERS_INDUSTRIES ei on ei.employer_id = e.id 
-- объединяем промежуточную табличу и таблицу сфер деятельности
left join public.INDUSTRIES i on i.id = ei.industry_id  
-- отбираем записи, где не указана сфера деятельности
where i.name is null           
'''

In [29]:
# результат запроса
result = pd.read_sql_query(query_5_4, connection)
result.iloc[0][0]

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''
select 
e.name
from public.EMPLOYERS e 
-- объединяем промежуточную таблицу и таблицу рабтодателей
left join public.EMPLOYERS_INDUSTRIES ei on ei.employer_id = e.id
-- объединяем промежуточную табличу и таблицу сфер деятельности
left join public.INDUSTRIES i on i.id = ei.industry_id
-- группируем по наименованию работодателя
group by 1 
-- отбираем только компании у которых 4 сферы деятельности
having count(i.name) = 4
-- сортируем по алфавиту по названию работодателя
order by 1 
-- пропускаем 2 первые записи в отборе
offset 2 
-- берем первую запись после пропуска
limit 1
'''

In [31]:
# результат запроса
result = pd.read_sql_query(query_5_5, connection)
print(result.iloc[0][0])

2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''
select 
-- считаем количесво работодателей
count(e.name)
from public.EMPLOYERS e 
-- объединяем промежуточную таблицу и таблицу рабтодателей
left join public.EMPLOYERS_INDUSTRIES ei on ei.employer_id = e.id
-- объединяем промежуточную табличу и таблицу сфер деятельности
left join public.INDUSTRIES i on i.id = ei.industry_id
-- отбираем тольок записи со сферой деятельности
-- 'Разработка программного обеспечения'
where i.name = 'Разработка программного обеспечения'
'''

In [33]:
# результат запроса
result = pd.read_sql_query(query_5_6, connection)
print(result.iloc[0][0])

3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-\
%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%\
D1%81%D1%81%D0%B8%D0%B8'

# получаем страницу по заданому адресу
response = requests.get(url)
# преобразуем ответ в объект BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')
# получаем набор строк таблицы с городами миллионниками
rows = soup.select(
    '#mw-content-text > div > table.standard.sortable > tbody > tr')
# получаем списко с названием городов миллионников
cities_list = []
for row in rows:
    columns = row.select('td')
    if len(columns) > 0:
        name = columns[1].select('a')[0].text.strip()
        cities_list.append(name)
# преобразуем список в строку
cities = ("'"+"','".join(cities_list)+"'")

In [35]:
# текст запроса
query_5_7 = f'''
(select
-- наименовнаие региона
a.name,
-- считаем количество вакансий
count(v.id) cnt 
from public.employers e
-- объединяем таблицы ваканси и работодателей
left join public.vacancies v on e.id = v.employer_id
-- объединяем таблицы вакансий и регионов
join public.areas a on a.id = v.area_id 
-- ограничиваем отбор по работодателям компанией 'Яндекс'
where e.name = 'Яндекс'
-- ограничиваем отбор по регионам городами миллионниками
and a.name in ({cities})
--группируем по наименованию рагиона
group by 1 )
-- объединяем таблицы по вертикали
union 
(select
-- задаем значение столбца
'Total',
-- считаем количество вакансий
count(v.id) cnt
from public.employers e
-- объединяем таблицы ваканси и работодателей
left join public.vacancies v on e.id = v.employer_id
-- объединяем таблицы вакансий и регионов
join public.areas a on a.id = v.area_id 
-- ограничиваем отбор по работодателям компанией 'Яндекс'
where e.name = 'Яндекс'
-- ограничиваем отбор по регионам городами миллионниками
and a.name in ({cities})
-- группируем по заданому значению столбца наименования региона
group by 1)
-- сортируем полученную объединённую таблицу по количеству вакансий
order by 2 
'''

In [36]:
# результат запроса
result = pd.read_sql_query(query_5_7, connection)
result

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***
### Выводы по анализу работодателей:

- *В рассматриваемый период наибольшее количество вакансий в технологических компаниях и банках.*
- *Вакансии распределяются по городам в количествах, пропорциональных населению. Что характерно в локализациях с названием государства, а не конкретного населенного пункта, наибольшее количество работодателей без вакансий.*
- *На первых местах по количеству регионов в которых работодатели публикуют вакансии ожидаемо находятся крупные глобально, в масштабах страны, представленные компании, такие как "Яндекс" и "Ростелеком".*
- *При рассмотрении распределения вакансий компании "Яндекс" по городам милионникам можно проследить тенденцию к корреляции количества вакансий с населением, 25% вакансий приходится на 16 городов милионников.*
- *У 17% работодателей не указана сфера деятельности, это комапнии с 1 редко больше вакансией, как правило не являющиеся слишком крупными и известными. Видимо по этим причинам не считали нужным указать сферу деятельности. Подавляющее большинство работодателей указывают 4 или 5 сфер деятельности. 
- *В качестве сферы деятельности "Разработка программного обеспечения" указана у 7,2%  работодателей.*

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''
select
-- считаем количество вакансий
count(v.name) 
from public.vacancies v 
-- ограничиваем выбор по условию содержания в названии
--  вакансии слов 'data' или 'данн'
where upper(v.name) like '%DATA%' or upper(v.name) like '%ДАНН%'
'''

In [38]:
# результат запроса
result = pd.read_sql_query(query_6_1, connection)
result.iloc[0][0]

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''
select
-- считаем количество вакансий
count(v.name)
from public.vacancies v
-- ограничиваем выбор по условию содержания в названии
-- вакансии слов 'data scientist', 'data science', 'исследователь данных',
-- 'machine learning', 'машинн%обучен%', переводя название вакансии в
-- нижний регистр для исключения дублирования случаев с различным написанием,
-- а так же 'ML', исключая возможность наличия 'HTML'. Дополнительно
-- ограничение по признаку  уровня Junior - содержания в названии в
-- названии вакансии слов 'junior', 'Нет опыта', 'Стажировка'.
where ((lower(v.name) like '%data scientist%' or lower(v.name) like \
'%data science%' or lower(v.name) like '%исследователь данных%' \
or lower(v.name) like '%machine learning%' or lower(v.name) like \
'%машинн%обучен%' or (v.name like '%ML%' and v.name not like '%HTML%')) \
and (lower(v.name) like '%junior%' or v.experience like '%Нет опыта%' \
or v.employment like 'Стажировка'))
'''

In [40]:
# результат запроса
result = pd.read_sql_query(query_6_2, connection)
result.iloc[0][0]

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
# текст запроса
query_6_3 = f'''
select
-- считаем количество вакансий
count(v.name)
from public.vacancies v
-- ограничиваем выбор по условию содержания в названии
-- вакансии слов 'data scientist', 'data science', 'исследователь данных',
-- 'machine learning', 'машинн%обучен%', переводя название вакансии в
-- нижний регистр для исключения дублирования случаев с различным написанием,
-- а так же 'ML', исключая возможность наличия 'HTML'. Дополнительно
-- ограничение по признаку ключевого навыка владение 'SQL', 'postgres'. 
where ((lower(v.name) like '%data scientist%' or lower(v.name) like \
'%data science%' or lower(v.name) like '%исследователь данных%' \
or lower(v.name) like '%machine learning%' or lower(v.name) like \
'%машинн%обучен%' or (v.name like '%ML%' and v.name not like '%HTML%')) \
and (v.key_skills like '%SQL%' or v.key_skills like '%postgres%'))
'''

In [42]:
# результат запроса
result = pd.read_sql_query(query_6_3, connection)
result.iloc[0][0]

201

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''
select
-- считаем количество вакансий
count(v.name)
from public.vacancies v
-- ограничиваем выбор по условию содержания в названии
-- вакансии слов 'data scientist', 'data science', 'исследователь данных',
-- 'machine learning', 'машинн%обучен%', переводя название вакансии в
-- нижний регистр для исключения дублирования случаев с различным написанием,
-- а так же 'ML', исключая возможность наличия 'HTML'. Дополнительно
-- ограничение по признаку ключевого навыка владение 'Python'. 
where ((lower(v.name) like '%data scientist%' or lower(v.name) like \
'%data science%' or lower(v.name) like '%исследователь данных%' \
or lower(v.name) like '%machine learning%' or lower(v.name) like \
'%машинн%обучен%' or (v.name like '%ML%' and v.name not like '%HTML%')) \
and (v.key_skills like '%Python%'))
'''

In [44]:
# результат запроса
result = pd.read_sql_query(query_6_4, connection)
result.iloc[0][0]

351

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''
select 
-- считаем среднее количество навыков, которое указывают в вакансиях
-- с округлением
Round((sum((LENGTH(v.key_skills)-LENGTH(replace(v.key_skills,chr(9),'')))\
                            +1)*1.0)/count(v.key_skills),2)
from public.vacancies v 
-- ограничиваем выбор по условию содержания в названии
-- вакансии слов 'data scientist', 'data science', 'исследователь данных',
-- 'machine learning', 'машинн%обучен%', переводя название вакансии в
-- нижний регистр для исключения дублирования случаев с различным написанием,
-- а так же 'ML', исключая возможность наличия 'HTML'. Исключаем 
-- пустое значение в поле навыков.
where ((lower(v.name) like '%data scientist%' or lower(v.name) like \
'%data science%' or lower(v.name) like '%исследователь данных%' \
or lower(v.name) like '%machine learning%' or lower(v.name) like \
'%машинн%обучен%' or (v.name like '%ML%' and v.name not like '%HTML%')) \
and v.key_skills is not null)
'''

In [46]:
# результат запроса
result = pd.read_sql_query(query_6_5, connection)
result.iloc[0][0]

6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса
query_6_6 = f'''
select 
distinct(v.experience),
-- считаем среднюю зарплату
round(avg(
case when v.salary_from is null then v.salary_to
     when v.salary_to is null then v.salary_from
     when v.salary_from is not null and v.salary_to is not null then
     ((coalesce(v.salary_from,0)+coalesce(v.salary_to,0))*1.0)/2 end),0) salary
from public.vacancies v 
-- ограничиваем выбор по условию содержания в названии
-- вакансии слов 'data scientist', 'data science', 'исследователь данных',
-- 'machine learning', 'машинн%обучен%', переводя название вакансии в
-- нижний регистр для исключения дублирования случаев с различным написанием,
-- а так же 'ML', исключая возможность наличия 'HTML'. Исключаем 
-- пустое значение в полях максималльной и минимальной ожидаемой зарплаты.
where ((lower(v.name) like '%data scientist%' or lower(v.name) like \
'%data science%' or lower(v.name) like '%исследователь данных%' \
or lower(v.name) like '%machine learning%' or lower(v.name) like \
'%машинн%обучен%' or (v.name like '%ML%' and v.name not like '%HTML%')) \
and (v.salary_from is not null or v.salary_to is not null))
group by 1
'''

In [48]:
# результат запроса
result = pd.read_sql_query(query_6_6, connection)
result

,experience,salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***
# Выводы по предметному анализу:

- *Из общего количества к данным имеет отношение 3,6% вакансий. А вот для начинающих датасаентистов всего 51 вканасия, что составляет 2,88% от вакансий, имеющих отношение к данным и 0,1% от общего количества опубликованных вакансий.*
- *Более распространены вакансии с ключевыми навыками владения SQL или Postgres 11,3% от вакансий относящихся к данным и 0,4% от общего количества и с клчевым навыком владения Python 19,8% от вакансий  относящихся к данным и 0,7% от общего количества вакансий.*
- *Среднее количество навыков, указываемых в вакансиях по DS 6.41.*
- *Средняя ожидаемая зарплата для датасаентистов достаточна высока, от 74643 руб. для вакансий без навыка работы до 243115 для опыта работы от 3 до 6 лет.*

# Общий вывод по проекту

   *В целом можно сделать следующие выводы на ресурсе НН представлен широкий спектр вакансий по многим городам и регионам. Вакансии размещаются фирами, функционирующими в различных сферах деятельности, но доля публикующих вакансии смещана в сторону технологических компаний. Тем не менне ра ресурсе представлены  работодатели с единичными вакансиями. Количество сфер деятельности по работодателям так же колеблется при средних значениях 4-5.*

   *Количество вакансий коррелирует с населением и промышленно-деловым потенциалом городов. Здесь было бы интересно оценить распределение средних зарплат и спектра вакансий по сферам деятельности в разрезе регионов.*
   
   *Вакансии представлены преимущественно позициями с не высокими требованиями к опыту и уровню зарплаты, что может говорить о использовании данной площадки в основном для поиска начинащих сотрудников, в то время как более опытные специалисты с более высокими запросами ищут работу по иным каналам. *
   
   *Доля вакансий для специалистов, работающих с данными, не велика и в целом составляет 3,6% от общего количества вакансий. А вот для начинающих датасаентистов всего 51 вканасия, что составляет 2,88% от вакансий, имеющих отношение к данным и 0,1% от общего количества опубликованных вакансий.*
   
   *Среднее количество навыков, указываемых в вакансиях по DS 6.41. При этом средняя предлагаемая зарплата для датасаентистов достаточна высока, от 74643 руб. для вакансий без навыка работы, и до 243115 для опыта работы от 3 до 6 лет. Что так же скорее говорит о поиске преимущественно начинающих специалистов.*